<a href="https://colab.research.google.com/github/olonok69/projects_ai/blob/main/Prompt_OLLama_DeepSeek_Phi3_5_LLama3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
%%capture output
! pip install -r /content/drive/MyDrive/models/requirements.txt -qqq

In [ ]:

%%capture output
! pip install langchain-ollama "ollama==0.4.2" -qqq

# Install Ollama
- curl https://ollama.ai/install.sh | sh

- Run server with: ollama serve &

- Pull model ollama: pull llama3:8b

In [ ]:
import pprint
import pandas as pd
import json

In [ ]:
session_data_path = "/content/drive/MyDrive/data/CSM/speaker_aggregated_info.json"  # For Recommendation engine
session_data_path = "/content/drive/MyDrive/data/CSM/badge_id_aggregated_results.json"
nomenclature_embeddings_path = (
    "/content/drive/MyDrive/data/CSM/cluster_numeculature.json"
)
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json"

csv_path = "/content/drive/MyDrive/data/CSM/new_claire_db_badge_cluster_data_with_aggregated_info_GIO_GT_LABELS.csv"
# csv_path = '/content/drive/MyDrive/data/CSM/20240512_new_labels_WITH_AGGINFO.csv'

In [ ]:
data = pd.read_csv(csv_path)
data.head()

In [ ]:
len(data)

In [ ]:
data.ClusterId.value_counts()

In [ ]:
df_networking = data[data["ClusterId"] == "Networking"]
df_learning = data[data["ClusterId"] == "Learning"]
df_searching = data[data["ClusterId"] == "Searching"]
df_sourcing_early = data[data["ClusterId"] == "Sourcing – Early"]
df_sourcing_inprocess = data[data["ClusterId"] == "Sourcing – In Process"]
df_sourcing_deciding = data[data["ClusterId"] == "Sourcing – Deciding"]

In [ ]:
example_networking = df_networking[0:1].to_dict()
example_learning = df_learning[0:1].to_dict()
example_searching = df_searching[0:1].to_dict()
example_sourcing_early = df_sourcing_early[0:1].to_dict()
example_sourcing_inprocess = df_sourcing_inprocess[0:1].to_dict()
example_sourcing_deciding = df_sourcing_deciding[0:1].to_dict()

In [ ]:
list_examples = [
    example_networking,
    example_learning,
    example_searching,
    example_sourcing_early,
    example_sourcing_inprocess,
    example_sourcing_deciding,
]

In [ ]:
examples = {}
for example in list_examples:

    examples[list(example["ClusterId"].values())[0]] = list(
        example["AggregatedInfo"].values()
    )[0]

In [ ]:
examples.keys()

In [ ]:
df_networking = df_networking[1:]
df_learning = df_learning[1:]
df_searching = df_searching[1:]
df_sourcing_early = df_sourcing_early[1:]
df_sourcing_inprocess = df_sourcing_inprocess[1:]
df_sourcing_deciding = df_sourcing_deciding[1:]

In [ ]:
with open(nomenclature_embeddings_path, "r") as f:
    nomenclatures = json.load(f)

In [ ]:
# nomenclatures

In [ ]:
text_example = df_searching[0:1]["AggregatedInfo"].values[0]

In [ ]:
# Function to generate the classification prompt
def generate_prompt(label_dict, examples, text_example):
    prompt = "Categories and Descriptions:\n\n"
    for label, description in label_dict.items():
        prompt += f"Category: {label} --> {description[len(label)+1:]}\n"

    prompt += "\nExamples: (1 text associated to a category)\n"
    for label, example in examples.items():
        prompt += f"Category: {label} --> {example}\n"

    prompt += "\nText to classify:\n"
    prompt += "----------------------------------------------------\n"
    prompt += text_example
    prompt += "----------------------------------------------------\n"
    prompt += "\n\n Only return a class label among these 6:\
        'Networking', 'Learning', 'Searching', 'Sourcing – Early', 'Sourcing – In Process', 'Sourcing – Deciding' "
    return prompt

In [ ]:
prompt = generate_prompt(nomenclatures, examples, text_example)
len(prompt)

In [ ]:
print(prompt)

# DeepSeek R1 Distill Qwen 1.5B

https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

https://ollama.com/library/deepseek-r1

ollama run deepseek-r1:1.5b

Q4_K - Medium


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage

In [ ]:
llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0.5,
    num_ctx=16000,
)
messages = [
    (
        "system",
        "You are a clever chatbot specialized in classify text.Only provide Label, no other Information. Let's think step by Step",
    ),
    ("human", prompt),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)
print("-" * 100)

In [ ]:
llm = ChatOllama(
    model="llama3.2:3b-instruct-q4_K_M",
    temperature=0.5,
    num_ctx=16000,
)
messages = [
    (
        "system",
        "You are a clever chatbot specialized in classify text.Only provide Label, no other Information. Let's think step by Step",
    ),
    ("human", prompt),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)
print("-" * 100)

In [ ]:
llm = ChatOllama(
    model="phi3.5",
    temperature=0.5,
    num_ctx=16000,
)
messages = [
    (
        "system",
        "You are a clever chatbot specialized in classify text.Only provide Label, no other Information. Let's think step by Step",
    ),
    ("human", prompt),
]
ai_msg = llm.invoke(messages)

print(ai_msg.content)
print("-" * 100)